# Load options data files
I've downloaded the options data for the `tickers` you requested for both `call` and `put` contracts for 45-day contracts starting on `2023-01-03`. Let's walk through the `call` contract data (the `put` contract procedure is the same; we change the `jld2` file name to `Options-Put-Daily-OHLC-45d-2023.jld2`)

## Setup

In [227]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`
  Activating project at `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Installed LinearOperators ─ v2.6.0
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
    Updating `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`
  [5c8ed15e] ↑ LinearOperators v2.5.2 ⇒ v2.6.0
Preco

log_return_matrix (generic function with 1 method)

In [226]:
function _loadcsvfile(path::String)::DataFrame
    return CSV.read(path, DataFrame);
end

_loadcsvfile (generic function with 1 method)

## Load `call` contract data
We downloaded data for 45-day contracts starting on `2023-01-03` and running through `2023`. The date information for each contract is given in the `call_contract_dates` variable. This is a `Tuple` that holds the day the contract was sold (`index = 1`), the day the contract expires (`index = 2`), and the number of days of the contract (`index=3`):

In [228]:
call_contract_dates = load(joinpath(_PATH_TO_DATA, "Options-Call-Daily-OHLC-45d-2023.jld2")) |> x -> x["dates"]

8-element Vector{Tuple{Date, Date, Int64}}:
 (Date("2023-01-03"), Date("2023-02-17"), 45)
 (Date("2023-02-22"), Date("2023-04-14"), 51)
 (Date("2023-04-17"), Date("2023-06-02"), 46)
 (Date("2023-06-05"), Date("2023-07-21"), 46)
 (Date("2023-07-24"), Date("2023-09-08"), 46)
 (Date("2023-09-11"), Date("2023-10-27"), 46)
 (Date("2023-10-30"), Date("2023-12-15"), 46)
 (Date("2023-12-18"), Date("2024-02-02"), 46)

To see the price information for each `call` contract, we load the `call_contract_data` dictionary. This data structure holds the price values for each ticker and each date combination. 

In [229]:
call_contract_data = load(joinpath(_PATH_TO_DATA, "Options-Call-Daily-OHLC-45d-2023.jld2")) |> x -> x["dataset"]

Dict{Tuple, NamedTuple} with 56 entries:
  ("AMD", Date("2023-04-17… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-06-05… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-01-03… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-02-22… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, 

In [230]:
call_contract_data

Dict{Tuple, NamedTuple} with 56 entries:
  ("AMD", Date("2023-04-17… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-06-05… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-01-03… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-02-22… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, 

We access this information by constructing a `tuple,` with the first item being the `ticker,` the second being the `sell` data, and the third item being the expiration date.
* For example, to look at the data for an `AMD` call sold on `2023-06-05` and expiring on `2023-07-21`, the `key` would be:

In [231]:
test_key = ("ADBE", Date("2023-01-03"), Date("2023-02-17"));

passing this `key` into the `call_contract_data` dictionary gives us a `dictionary` holding the `OHLC` data for several strike prices:

In [232]:
ADBE = call_contract_data[test_key] |> x-> x[:data]

Dict{Float64, Union{Nothing, DataFrame}} with 13 entries:
  315.0 => 19×8 DataFrame…
  305.0 => 3×8 DataFrame…
  345.0 => 32×8 DataFrame…
  310.0 => 9×8 DataFrame…
  325.0 => 22×8 DataFrame…
  365.0 => 33×8 DataFrame…
  340.0 => 29×8 DataFrame…
  350.0 => 33×8 DataFrame…
  335.0 => 29×8 DataFrame…
  330.0 => 30×8 DataFrame…
  355.0 => 33×8 DataFrame…
  320.0 => 18×8 DataFrame…
  360.0 => 33×8 DataFrame…

In [235]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-12-01-2023.jld2")) |> x-> x["dataset"];

In [236]:
maximum_number_trading_days = original_dataset["MSFT"] |> nrow;

dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers);

startdate = Date(2023,01,03);
MSFT_dataset = dataset["MSFT"];
MSFT_df = filter(:timestamp => x-> x >= startdate, MSFT_dataset);

In [239]:
function option_value(market_df::DataFrame, options_df::DataFrame, strike_price::Float64)
    last_day = maximum(options_df[:, :timestamp])
    last_day_market_info = market_df[market_df[:, :timestamp] .== last_day, :]

    last_day_market_price = last_day_market_info[1, :close]
    if last_day_market_price < strike_price
        # If the option expires worthless return 1.0, otherwise return 0.0
        return 1.0
    end
end


option_value (generic function with 1 method)

In [802]:
function get_good_calls(ticker::String, AssetDict::Dict{Float64, Union{Nothing, DataFrame}})
    good_calls = []
    for (strike_price, options_df) in AssetDict
        result = option_value(tech_price_df[ticker], options_df, strike_price)
        sale_date = Date(minimum(options_df[:, :timestamp]))
        market_df = tech_price_df[ticker]
        last_day_market_info = market_df[market_df[:, :timestamp] .== minimum(options_df[:, :timestamp]), :]
        last_day_market_price = last_day_market_info[1, :close]
        
        if result !== nothing
            premium = round(options_df[1, :]["volume_weighted_average_price"] * 100, digits=2)
            call_description = "On $sale_date when $ticker is trading at $last_day_market_price, sell a call with a strike price $strike_price for a premium $premium."
            push!(good_calls, call_description, premium) 
        end
    end

    return good_calls
end

get_good_calls (generic function with 1 method)

In [804]:
tech_portfolio = ["AMD", "AAPL", "MSFT", "NVDA", "ADBE", "SPY"]
tech_contracts = Dict()

for (key, value) in call_contract_data
    ticker, date = key
    if ticker in tech_portfolio
        tech_contracts[key] = value
    end
end

In [805]:
tech_contracts

Dict{Any, Any} with 42 entries:
  ("ADBE", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-04-17… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-06-05… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-09-11… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-01-03… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame

In [806]:
call_contract_dates[1:3]

3-element Vector{Tuple{Date, Date, Int64}}:
 (Date("2023-01-03"), Date("2023-02-17"), 45)
 (Date("2023-02-22"), Date("2023-04-14"), 51)
 (Date("2023-04-17"), Date("2023-06-02"), 46)

In [807]:
MyFirmParametersDataSet() = _loadcsvfile(joinpath(_PATH_TO_DATA, "Parameters-SP500-2018-2022.csv"));

In [808]:
params = MyFirmParametersDataSet()

Row,index,ticker,drift,volatility
,Int64,String7,Float64,Float64
1,1,MMM,-0.0822692,0.238729
2,2,AOS,0.0441975,0.266025
3,3,ABT,0.151149,0.230271
4,4,ABBV,0.118596,0.266743
5,6,ACN,0.178695,0.250005
6,7,ATVI,0.0745844,0.305086
7,8,ADM,0.161607,0.237929
8,9,ADBE,0.171392,0.307362
9,10,AAP,0.0890969,0.307218


In [809]:
tech_portfolio = ["AMD", "AAPL", "MSFT", "IBM", "ADBE", "AMZN"]
tech_params = filter(row -> row[:ticker] in tech_portfolio, params)

Row,index,ticker,drift,volatility
,Int64,String7,Float64,Float64
1,9,ADBE,0.171392,0.307362
2,11,AMD,0.460484,0.468767
3,27,AMZN,0.144741,0.303115
4,46,AAPL,0.33739,0.276418
5,241,IBM,-0.0181696,0.241395
6,315,MSFT,0.268955,0.239654


In [810]:
tech_params[!, "mod_sharpe"] = tech_params[!, "drift"] ./ tech_params[!, "volatility"]

6-element Vector{Float64}:
  0.5576236300689372
  0.9823318860023869
  0.4775115125933747
  1.220581292852611
 -0.07526920226807625
  1.1222653702327074

In [811]:
tech_params

Row,index,ticker,drift,volatility,mod_sharpe
,Int64,String7,Float64,Float64,Float64
1,9,ADBE,0.171392,0.307362,0.557624
2,11,AMD,0.460484,0.468767,0.982332
3,27,AMZN,0.144741,0.303115,0.477512
4,46,AAPL,0.33739,0.276418,1.22058
5,241,IBM,-0.0181696,0.241395,-0.0752692
6,315,MSFT,0.268955,0.239654,1.12227


In [812]:
p = pretty_table(tech_params)

┌───────┬─────────┬────────────┬────────────┬────────────┐
│ index │  ticker │      drift │ volatility │ mod_sharpe │
│ Int64 │ String7 │    Float64 │    Float64 │    Float64 │
├───────┼─────────┼────────────┼────────────┼────────────┤
│     9 │    ADBE │   0.171392 │   0.307362 │   0.557624 │
│    11 │     AMD │   0.460484 │   0.468767 │   0.982332 │
│    27 │    AMZN │   0.144741 │   0.303115 │   0.477512 │
│    46 │    AAPL │    0.33739 │   0.276418 │    1.22058 │
│   241 │     IBM │ -0.0181696 │   0.241395 │ -0.0752692 │
│   315 │    MSFT │   0.268955 │   0.239654 │    1.12227 │
└───────┴─────────┴────────────┴────────────┴────────────┘


In [813]:
tech_portfolio = ["ADBE", "AMD", "MSFT", "AAPL", "IBM", "AMZN"]
tech_calls = Dict()
for (k, v) in call_contract_data
    if k[1] in tech_portfolio
        tech_calls[k] = v
    end
end

In [814]:
tech_calls

Dict{Any, Any} with 28 entries:
  ("ADBE", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-04-17… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-09-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-01-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-02-22… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame

In [815]:
adbe_calls = Dict(key => value for (key, value) in tech_calls if key[1] == "ADBE")
amd_calls = Dict(key => value for (key, value) in tech_calls if key[1] == "AMD")
msft_calls = Dict(key => value for (key, value) in tech_calls if key[1] == "MSFT")
aapl_calls = Dict(key => value for (key, value) in tech_calls if key[1] == "AAPL")

Dict{Tuple{String, Date, Date}, NamedTuple{(:data, :header), Tuple{Dict{Float64, Union{Nothing, DataFrame}}, Dict{Float64, Any}}}} with 7 entries:
  ("AAPL", Date("2023-02-2… => (data = Dict(155.0=>31×8 DataFrame…
  ("AAPL", Date("2023-04-1… => (data = Dict(150.0=>33×8 DataFrame…
  ("AAPL", Date("2023-09-1… => (data = Dict(150.0=>30×8 DataFrame…
  ("AAPL", Date("2023-06-0… => (data = Dict(150.0=>33×8 DataFrame…
  ("AAPL", Date("2023-10-3… => (data = Dict(150.0=>28×8 DataFrame…
  ("AAPL", Date("2023-07-2… => (data = Dict(200.0=>31×8 DataFrame…
  ("AAPL", Date("2023-01-0… => (data = Dict(100.0=>31×8 DataFrame…

In [816]:
call_tickers = ["ADBE", "AMD", "MSFT", "AAPL"]

4-element Vector{String}:
 "ADBE"
 "AMD"
 "MSFT"
 "AAPL"

In [817]:
function calculate_share_costs(call_tickers, tech_price_df)
    collateral_df = DataFrame(Ticker = String[], Cost_of_100_shares = Float64[])

    for ticker in call_tickers
        cost = tech_price_df[ticker][1, :]["volume_weighted_average_price"] * 100
        push!(collateral_df, (ticker, cost))
    end

    return collateral_df
end

calculate_share_costs (generic function with 1 method)

In [818]:
collateral_df = calculate_share_costs(call_tickers, tech_price_df)

Row,Ticker,Cost_of_100_shares
,String,Float64
1,ADBE,33741.6
2,AMD,6455.91
3,MSFT,23983.9
4,AAPL,12572.5


In [819]:
#assuming one active call per asset
total_collateral_cost = sum(collateral_df[!, "Cost_of_100_shares"])

76753.94

In [821]:
function get_price_data(tickers::Array{String,1}, original_dataset::Dict)
    price_data = Dict{String, DataFrame}()

    for ticker in tickers
        maximum_number_trading_days = nrow(original_dataset[ticker])
        dataset = Dict{String, DataFrame}()

        for (t, data) in original_dataset
            if nrow(data) == maximum_number_trading_days
                dataset[t] = data
            end
        end
        startdate = Date(2023, 01, 03)
        asset_data = filter(row -> row[:timestamp] >= startdate, dataset[ticker])

        price_data[ticker] = asset_data
    end

    return price_data
end

get_price_data (generic function with 2 methods)

In [822]:
tech_price_df= get_price_data(tech_portfolio, original_dataset)

Dict{String, DataFrame} with 6 entries:
  "MSFT" => 231×8 DataFrame…
  "AMZN" => 231×8 DataFrame…
  "AMD"  => 231×8 DataFrame…
  "ADBE" => 231×8 DataFrame…
  "IBM"  => 231×8 DataFrame…
  "AAPL" => 231×8 DataFrame…

# Calculate the contracts that will expire worthless and their premiums

In [823]:
#partially generated with GenAI and modified
function optimal_covered_calls(tickers::Array{String, 1}, call_contract_data::Dict)
    summary_dict = Dict()
    date_ranges = [
        (Date(2023, 1, 3), Date(2023, 2, 17)),
        (Date(2023, 2, 22), Date(2023, 4, 14)),
        (Date(2023, 4, 17), Date(2023, 6, 2)),
        (Date(2023, 6, 5), Date(2023, 7, 21)),
        (Date(2023, 7, 24), Date(2023, 9, 8)),
        (Date(2023, 9, 11), Date(2023, 10, 27))
    ]

    for ticker in tickers
        ticker_summary = []
        for (idx, (start_date, end_date)) in enumerate(date_ranges)
            contract_key = (ticker, start_date, end_date)
            if haskey(call_contract_data, contract_key)
                options_data = call_contract_data[contract_key] |> x -> x[:data]
                good_calls = get_good_calls(ticker, options_data)
                push!(ticker_summary, (idx, good_calls))
            else
                println("No contract data for key: ", contract_key)
            end
        end
        if !isempty(ticker_summary)
            summary_dict[ticker] = ticker_summary
        end
    end
    return summary_dict
end


optimal_covered_calls (generic function with 1 method)

In [842]:
optimal_covered_calls_dict = optimal_covered_calls(call_tickers, call_contract_data)

Dict{Any, Any} with 4 entries:
  "MSFT" => Any[(1, Any["On 2023-01-03 when MSFT is trading at 239.58, sell a c…
  "AMD"  => Any[(1, Any["On 2023-01-03 when AMD is trading at 64.02, sell a cal…
  "ADBE" => Any[(1, Any["On 2023-01-03 when ADBE is trading at 336.92, sell a c…
  "AAPL" => Any[(1, Any["On 2023-01-03 when AAPL is trading at 125.07, sell a c…

## All prices within the contracts fall between 1 STD and 2 STD from the upper bound of the mean for the GBM ran on these assets.

#### This means we are between 68% and 95% confident we will not be exercised on for these options.

In [825]:
optimal_covered_calls_dict["MSFT"]

6-element Vector{Any}:
 (1, Any["On 2023-01-03 when MSFT is trading at 239.58, sell a call with a strike price 260.0 for a premium 471.43.", 471.43, "On 2023-01-03 when MSFT is trading at 239.58, sell a call with a strike price 265.0 for a premium 308.08.", 308.08, "On 2023-01-03 when MSFT is trading at 239.58, sell a call with a strike price 270.0 for a premium 262.69.", 262.69])
 (2, Any[])
 (3, Any[])
 (4, Any["On 2023-06-05 when MSFT is trading at 335.94, sell a call with a strike price 345.0 for a premium 708.48.", 708.48, "On 2023-06-05 when MSFT is trading at 335.94, sell a call with a strike price 365.0 for a premium 190.89.", 190.89, "On 2023-06-05 when MSFT is trading at 335.94, sell a call with a strike price 350.0 for a premium 510.23.", 510.23, "On 2023-06-05 when MSFT is trading at 335.94, sell a call with a strike price 355.0 for a premium 385.68.", 385.68, "On 2023-06-05 when MSFT is trading at 335.94, sell a call with a strike price 360.0 for a premium 268.49.", 268.49

In [826]:
optimal_covered_calls_dict["AMD"]

6-element Vector{Any}:
 (1, Any["On 2023-01-03 when AMD is trading at 64.02, sell a call with a strike price 95.0 for a premium 15.36.", 15.36, "On 2023-01-03 when AMD is trading at 64.02, sell a call with a strike price 90.0 for a premium 28.59.", 28.59, "On 2023-01-03 when AMD is trading at 64.02, sell a call with a strike price 85.0 for a premium 46.33.", 46.33, "On 2023-01-03 when AMD is trading at 64.02, sell a call with a strike price 80.0 for a premium 92.14.", 92.14])
 (2, Any["On 2023-03-03 when AMD is trading at 81.52, sell a call with a strike price 95.0 for a premium 84.52.", 84.52, "On 2023-03-06 when AMD is trading at 81.16, sell a call with a strike price 105.0 for a premium 21.91.", 21.91, "On 2023-03-03 when AMD is trading at 81.52, sell a call with a strike price 100.0 for a premium 41.0.", 41.0])
 (3, Any["On 2023-04-17 when AMD is trading at 89.87, sell a call with a strike price 120.0 for a premium 20.75.", 20.75])
 (4, Any["On 2023-06-05 when AMD is trading at 117

In [827]:
optimal_covered_calls_dict["ADBE"]

6-element Vector{Any}:
 (1, Any["On 2023-01-03 when ADBE is trading at 336.92, sell a call with a strike price 365.0 for a premium 862.37.", 862.37, "On 2023-01-03 when ADBE is trading at 336.92, sell a call with a strike price 360.0 for a premium 944.04.", 944.04])
 (2, Any["On 2023-03-02 when ADBE is trading at 333.5, sell a call with a strike price 380.0 for a premium 318.5.", 318.5])
 (3, Any[])
 (4, Any[])
 (5, Any[])
 (6, Any["On 2023-09-11 when ADBE is trading at 564.5, sell a call with a strike price 570.0 for a premium 2381.0.", 2381.0, "On 2023-09-12 when ADBE is trading at 542.21, sell a call with a strike price 545.0 for a premium 2946.86.", 2946.86, "On 2023-09-11 when ADBE is trading at 564.5, sell a call with a strike price 560.0 for a premium 2770.0.", 2770.0, "On 2023-09-12 when ADBE is trading at 542.21, sell a call with a strike price 550.0 for a premium 2482.43.", 2482.43, "On 2023-09-12 when ADBE is trading at 542.21, sell a call with a strike price 530.0 for a pre

In [828]:
optimal_covered_calls_dict["AAPL"]

6-element Vector{Any}:
 (1, Any["On 2023-01-03 when AAPL is trading at 125.07, sell a call with a strike price 155.0 for a premium 38.94.", 38.94])
 (2, Any["On 2023-03-03 when AAPL is trading at 151.03, sell a call with a strike price 180.0 for a premium 1.0.", 1.0, "On 2023-03-02 when AAPL is trading at 145.91, sell a call with a strike price 175.0 for a premium 10.5.", 10.5, "On 2023-03-03 when AAPL is trading at 151.03, sell a call with a strike price 170.0 for a premium 29.88.", 29.88])
 (3, Any["On 2023-04-18 when AAPL is trading at 166.47, sell a call with a strike price 195.0 for a premium 10.0.", 10.0, "On 2023-04-17 when AAPL is trading at 165.23, sell a call with a strike price 190.0 for a premium 18.5.", 18.5, "On 2023-04-17 when AAPL is trading at 165.23, sell a call with a strike price 185.0 for a premium 37.51.", 37.51])
 (4, Any["On 2023-06-05 when AAPL is trading at 179.58, sell a call with a strike price 200.0 for a premium 85.99.", 85.99, "On 2023-06-05 when AAPL is 

In [829]:
optimal_covered_calls_dict["AAPL"][4][2][1]

"On 2023-06-05 when AAPL is trading at 179.58, sell a call with a strike price 200.0 for a premium 85.99."

In [830]:
function extract_premiums_from_vector(vector_data::Vector{Any})
    premiums = Float64[]

    #GenAI created for loop to iterate into the vector data bc idk how to do this in Julia
    for (_, call_descriptions) in vector_data
        for i in 2:2:length(call_descriptions)
            push!(premiums, call_descriptions[i])
        end
    end

    return premiums
end

extract_premiums_from_vector (generic function with 1 method)

In [831]:
total_premium = 0
for ticker in call_tickers
    total_premium += sum(extract_premiums_from_vector(optimal_covered_calls_dict[ticker]))
end
total_premium

67558.53

## Return generated by the options strategy alone (assuming 1 optimal contract per period)

In [832]:
premium_return = 1+total_premium/total_collateral_cost

1.8801962479059706

In [833]:
return_reallocation = 6.09643e7/4.22281e7

1.4436903388975588

In [834]:
return_overall = (6.09643e7 + total_collateral_cost + total_premium)/(4.22281e7 + total_collateral_cost)

1.4444822940806967

In [944]:
yield_df = DataFrame(CoveredCallYield=premium_return, ReallocationYield=return_reallocation, OverallYield=return_overall)
pretty_table(yield_df)

┌──────────────────┬───────────────────┬──────────────┐
│ CoveredCallYield │ ReallocationYield │ OverallYield │
│          Float64 │           Float64 │      Float64 │
├──────────────────┼───────────────────┼──────────────┤
│           1.8802 │           1.44369 │      1.44448 │
└──────────────────┴───────────────────┴──────────────┘


# Computing exercise probabilities

In [874]:
typeof(tech_params)

DataFrame

In [872]:
tech_params[tech_params.ticker .== "MSFT", :volatility][1]

0.23965404099633378

In [929]:
# modified functions defined above to create dataframes instead of vectors of strings
using Distributions
function get_good_calls_dataframe(ticker::String, AssetDict::Dict{Float64, Union{Nothing, DataFrame}}, tech_params::DataFrame)
    good_calls_df = DataFrame(Date=Date[], Ticker=String[], MarketPrice=Float64[], StrikePrice=Float64[], Std=Float64[], Percentage=Float64[])
    for (strike_price, options_df) in AssetDict
        result = option_value(tech_price_df[ticker], options_df, strike_price)
        if result != nothing && !isempty(options_df)
            sale_date = Date(minimum(options_df[:, :timestamp]))
            market_df = tech_price_df[ticker]
            last_day_market_info = market_df[market_df[:, :timestamp] .== minimum(options_df[:, :timestamp]), :]
            last_day_market_price = last_day_market_info[1, :close]
            vol = tech_params[tech_params.ticker .== ticker, :volatility][1]*last_day_market_price
            std_away = abs(last_day_market_price - strike_price) / vol
            calc_std = std_dev -> (cdf(Normal(0, 1), std_dev) - cdf(Normal(0, 1), -std_dev)) * 100
            percentage_within_std = calc_std(std_away)
            push!(good_calls_df, (sale_date, ticker, last_day_market_price, strike_price, std_away, percentage_within_std))
        end
    end

    return good_calls_df
end

get_good_calls_dataframe (generic function with 2 methods)

In [930]:
function dataframe_gen_calls(tickers::Array{String, 1}, call_contract_data::Dict, params::DataFrame)
    summary_dict = Dict()
    date_ranges = [
        (Date(2023, 1, 3), Date(2023, 2, 17)),
        (Date(2023, 2, 22), Date(2023, 4, 14)),
        (Date(2023, 4, 17), Date(2023, 6, 2)),
        (Date(2023, 6, 5), Date(2023, 7, 21)),
        (Date(2023, 7, 24), Date(2023, 9, 8)),
        (Date(2023, 9, 11), Date(2023, 10, 27))
    ]

    for ticker in tickers
        dfs = []
        for (idx, (start_date, end_date)) in enumerate(date_ranges)
            contract_key = (ticker, start_date, end_date)
            if haskey(call_contract_data, contract_key)
                options_data = call_contract_data[contract_key] |> x -> x[:data]
                good_calls = get_good_calls_dataframe(ticker, options_data, params)
                push!(dfs, (idx, good_calls))
            else
                println("No contract data for key: ", contract_key)
            end
        end
        if !isempty(dfs)
            summary_dict[ticker] = dfs
        end
    end
    return summary_dict
end

dataframe_gen_calls (generic function with 2 methods)

In [931]:
dfs= dataframe_gen_calls(call_tickers, call_contract_data,  tech_params)

Dict{Any, Any} with 4 entries:
  "MSFT" => Any[(1, 3×6 DataFrame…
  "AMD"  => Any[(1, 4×6 DataFrame…
  "ADBE" => Any[(1, 2×6 DataFrame…
  "AAPL" => Any[(1, 1×6 DataFrame…

In [932]:
dfs["AAPL"]

6-element Vector{Any}:
 (1, 1×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std      Percentage 
     │ Date        String  Float64      Float64      Float64  Float64    
─────┼───────────────────────────────────────────────────────────────────
   1 │ 2023-01-03  AAPL         125.07        155.0  0.86574     61.3367)
 (2, 3×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-03-03  AAPL         151.03        180.0  0.693936     51.2277
   2 │ 2023-03-02  AAPL         145.91        175.0  0.721261     52.9251
   3 │ 2023-03-03  AAPL         151.03        170.0  0.4544       35.0459)
 (3, 3×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────

In [933]:
dfs["ADBE"]

6-element Vector{Any}:
 (1, 2×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-01-03  ADBE         336.92        365.0  0.271156     21.3729
   2 │ 2023-01-03  ADBE         336.92        360.0  0.222874     17.6366)
 (2, 1×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-03-02  ADBE          333.5        380.0  0.453635     34.9908)
 (3, 0×6 DataFrame
 Row │ Date  Ticker  MarketPrice  StrikePrice  Std      Percentage 
     │ Date  String  Float64      Float64      Float64  Float64    
─────┴─────────────────────────────────────────────────────────────)
 (4, 0×6 DataFrame
 Row │ Date  Ticker  M

Here is a vector of Dataframes, each which takes the options that we expect to expire worthless and analyzes their market price and strike price. We use this to understand how far from the expected value of the market price are we willing to sell calls at. In a sense, we want to know how many standard deviations away from the expected market price is our strike price to sell calls that will expire OTM.

In [939]:
dfs["MSFT"]

6-element Vector{Any}:
 (1, 3×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-01-03  MSFT         239.58        260.0  0.355648     27.7896
   2 │ 2023-01-03  MSFT         239.58        265.0  0.442731     34.204
   3 │ 2023-01-03  MSFT         239.58        270.0  0.529815     40.3759)
 (2, 0×6 DataFrame
 Row │ Date  Ticker  MarketPrice  StrikePrice  Std      Percentage 
     │ Date  String  Float64      Float64      Float64  Float64    
─────┴─────────────────────────────────────────────────────────────)
 (3, 0×6 DataFrame
 Row │ Date  Ticker  MarketPrice  StrikePrice  Std      Percentage 
     │ Date  String  Float64      Float64      Float64  Float64    
─────┴─────────────────────────────────────────────────────────────)
 (4, 5×6 DataFrame
 Row │ Date        Ticker  MarketPrice  Stri

In [940]:
dfs["AMD"]

6-element Vector{Any}:
 (1, 4×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-01-03  AMD           64.02         95.0  1.03231      69.8072
   2 │ 2023-01-03  AMD           64.02         90.0  0.865699     61.3345
   3 │ 2023-01-03  AMD           64.02         85.0  0.69909      51.5504
   4 │ 2023-01-03  AMD           64.02         80.0  0.532481     40.5607)
 (2, 3×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-03-03  AMD           81.52         95.0  0.352752     27.5725
   2 │ 2023-03-06  AMD           81.16        105.0  0.626625     46.9095
   3 │ 2023-03-03  AMD           81.52      

sanity check -- it works!

In [935]:
@assert sum(nrow(df[2]) for df in dfs["MSFT"]) == length(extract_premiums_from_vector(optimal_covered_calls_dict["MSFT"]))

In [966]:
function exercise_prob(dfs::Vector{Any})
    total_percentage = 0.0
    count = 1
    
    for df in dfs
        df[2][count]
        total_percentage += sum(df[:, :Percentage])
        count += 1
    end
    
    return total_percentage / count
end

exercise_prob (generic function with 3 methods)

In [991]:
dfs

Dict{Any, Any} with 4 entries:
  "MSFT" => Any[(1, 3×6 DataFrame…
  "AMD"  => Any[(1, 4×6 DataFrame…
  "ADBE" => Any[(1, 2×6 DataFrame…
  "AAPL" => Any[(1, 1×6 DataFrame…

In [992]:
dfs["MSFT"]

6-element Vector{Any}:
 (1, 3×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-01-03  MSFT         239.58        260.0  0.355648     27.7896
   2 │ 2023-01-03  MSFT         239.58        265.0  0.442731     34.204
   3 │ 2023-01-03  MSFT         239.58        270.0  0.529815     40.3759)
 (2, 0×6 DataFrame
 Row │ Date  Ticker  MarketPrice  StrikePrice  Std      Percentage 
     │ Date  String  Float64      Float64      Float64  Float64    
─────┴─────────────────────────────────────────────────────────────)
 (3, 0×6 DataFrame
 Row │ Date  Ticker  MarketPrice  StrikePrice  Std      Percentage 
     │ Date  String  Float64      Float64      Float64  Float64    
─────┴─────────────────────────────────────────────────────────────)
 (4, 5×6 DataFrame
 Row │ Date        Ticker  MarketPrice  Stri

In [1007]:
dfs["MSFT"][4]

(4, 5×6 DataFrame
 Row │ Date        Ticker  MarketPrice  StrikePrice  Std       Percentage 
     │ Date        String  Float64      Float64      Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────
   1 │ 2023-06-05  MSFT         335.94        345.0  0.112533     8.95996
   2 │ 2023-06-05  MSFT         335.94        365.0  0.360952    28.1864
   3 │ 2023-06-05  MSFT         335.94        350.0  0.174638    13.8636
   4 │ 2023-06-05  MSFT         335.94        355.0  0.236743    18.7144
   5 │ 2023-06-05  MSFT         335.94        360.0  0.298847    23.4943)

In [1019]:
function average_exercise_prob(tickers::Array{String, 1}, dfs::Dict{Any, Any})
    exercise_df = DataFrame(Ticker=String[], AverageExerciseProbability=Float64[])
    for ticker in tickers
        total_percentage = 0.0
        count = 0
        for tuple in dfs[ticker]
            df = tuple[2]
            total_percentage += sum(df[:, :Percentage])
            count += nrow(df)
        end
        contract_percentage = total_percentage / count
        push!(exercise_df, (ticker, contract_percentage))
    end
    return exercise_df
end

average_exercise_prob (generic function with 2 methods)

In [1021]:
pretty_table(average_exercise_prob(call_tickers, dfs))

┌────────┬────────────────────────────┐
│ Ticker │ AverageExerciseProbability │
│ String │                    Float64 │
├────────┼────────────────────────────┤
│   ADBE │                    12.0085 │
│    AMD │                    27.9886 │
│   MSFT │                    20.2106 │
│   AAPL │                    28.5885 │
└────────┴────────────────────────────┘
